In [1]:
from bs4 import BeautifulSoup
from pathlib import Path
from datetime import datetime, timedelta
import json

In [2]:
data_fields = {
    9: "staff.on",
    10: "staff.off",
    11: "student.on",
    12: "student.off",
}
text_fields = {
    1: "Staff",
    2: "Students",
    4: "On campus *",
    5: "Off campus **",
    6: "On campus *",
    7: "Off campus **",
    8: "New cases in last counted 24 hour period ***",
    13: "New cases in last counted 7 day period ***",
    18: "Total cases since 28 Sept 2020 (start of Term 1)",
}
def parse_file(fh):
    soup = BeautifulSoup(fh, 'html.parser')
    table = soup.select_one('#current-confirmed-cases-covid-19 > div.site-content.wrapper > div > div > div > article > div > table')
    data = {}
    for i, tag in enumerate(table.find_all(["td","th"])):
        if i in text_fields:
            assert(tag.string == text_fields[i])
        elif i in data_fields:
            data[data_fields[i]] = int(tag.string)
            
    return data

In [3]:
p = Path('data')
last_data = None
dataset_names = ['staff.on', 'staff.off', 'student.on', 'student.off']
datasets = {}
for file in p.glob("../data/covid-*.html"):
    fh = open(file)
    file_date = datetime.strptime(file.name, "covid-%Y-%m-%dT%H-%M-%S.html").date() - timedelta(days = 1)
    data = parse_file(fh)
    if data != last_data:
        for n in dataset_names:
            ds = datasets.setdefault(n, [])
            ds.append((file_date.strftime("%Y-%m-%d"), data[n]))
            ds.append(((file_date + timedelta(days=1)).strftime("%Y-%m-%d"), data[n]*1.1))
        last_data = data
        
ofh = open("../data/covid.json", "w")
json.dump(datasets, ofh, sort_keys=True, indent=4)
ofh.close()